In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
%matplotlib inline

In [14]:
from google.colab import files
uploaded = files.upload()

Saving imp.csv to imp.csv


In [4]:
dataset = pd.read_csv('imp.csv',sep=',')

In [5]:
cr = (dataset['Количество лидов']/dataset['Посетители'])*100
month = dataset['Интервал дат визита'].str.split('-')
dataset['CR'] = cr
X = dataset.drop(['Количество лидов','Интервал дат визита'], axis=1)
Y = dataset['Количество лидов']
x_train, x_test, y_train, y_test = train_test_split(X, Y, train_size=0.80, random_state = 0)

In [39]:
data = [['3637','3159','6852','97','13','4','6','7.1','2.009165']]
df = pd.DataFrame(data, columns=['Визиты','Посетители','Просмотры','Облачность','Осадки','Днем','Ночью','Ветер','CR'])

### **Разные модели**

**Бэггинг**

In [60]:
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
tree_model = BaggingRegressor(base_estimator=DecisionTreeRegressor(max_depth=9), n_estimators=10, random_state=0, bootstrap=True)
tree_model.fit(x_train, y_train)
tree_result = tree_model.predict(x_test)

mse = mean_squared_error(y_test, tree_result)
mae = mean_absolute_error(y_test, tree_result) 

cross_v = cross_val_score(tree_model, X, Y, cv=20)

print("RMSE: {}".format(np.sqrt(mse)))
print("MAE: {}".format(mae))
print("Predict: {}".format(tree_model.predict(df)[0]))

print(cross_v)

RMSE: 7.491536136306644
MAE: 4.13032821602266
Predict: 63.2
[0.82597211 0.9731229  0.93375584 0.98416448 0.78838456 0.74672024
 0.7900819  0.93274921 0.88632607 0.87130934 0.96507525 0.86174961
 0.98659007 0.97587122 0.9575642  0.97703435 0.95793273 0.99153743
 0.98870555 0.98833438]


**Сверхслучайные деревья** - шумов минимум, по показателям норм - поменял на них

In [61]:
from sklearn.ensemble import ExtraTreesRegressor
reg = ExtraTreesRegressor(n_estimators=8, random_state=0)
reg.fit(x_train, y_train)
reg_result = reg.predict(x_test)
mse_r = mean_squared_error(y_test, reg_result)
mae_r = mean_absolute_error(y_test, reg_result) 
cross_v2 = cross_val_score(reg, X, Y, cv=20)
print("RMSE: {}".format(np.sqrt(mse_r)))
print("MAE: {}".format(mae_r))
print("Predict: {}".format(reg.predict(df)[0]))
print(cross_v2)

RMSE: 3.642955213173116
MAE: 2.821759259259259
Predict: 66.875
[0.91445559 0.88956984 0.91412173 0.99070427 0.87282241 0.83485635
 0.91223379 0.98151625 0.91051517 0.94152016 0.99122461 0.89882251
 0.99183642 0.97666213 0.98979788 0.97928726 0.96917766 0.98509976
 0.9793164  0.96434197]


**Бустинг**
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html

In [62]:
from sklearn.ensemble import GradientBoostingRegressor
reg2 = GradientBoostingRegressor(random_state=0)
reg2.fit(x_train, y_train)
reg2_result=reg2.predict(x_test)
mse_r2 = mean_squared_error(y_test, reg2_result)
mae_r2 = mean_absolute_error(y_test, reg2_result) 
cross_v3 = cross_val_score(reg, X, Y, cv=20)
print("RMSE: {}".format(np.sqrt(mse_r2)))
print("MAE: {}".format(mae_r2))
print("Predict: {}".format(reg.predict(df)[0]))
print(cross_v3)

RMSE: 5.6751839870178715
MAE: 3.4675484174055478
Predict: 66.875
[0.91445559 0.88956984 0.91412173 0.99070427 0.87282241 0.83485635
 0.91223379 0.98151625 0.91051517 0.94152016 0.99122461 0.89882251
 0.99183642 0.97666213 0.98979788 0.97928726 0.96917766 0.98509976
 0.9793164  0.96434197]


кросс валидация
r2